# A Brief History of Async

## Concurrency

Concurrency == simulating parallelism by switching context.

![concurrency](concurrency.png)

### Sequential execution

In [40]:
import httpx


def job(n):
    print(f"--- request {n} sent")
    httpx.get(f"https://example.com/{n}")
    print(f"--- response {n} received")

In [41]:
%%time

job(1)
job(2)

--- request 1 sent
--- response 1 received
--- request 2 sent
--- response 2 received
CPU times: user 47.7 ms, sys: 7.25 ms, total: 54.9 ms
Wall time: 2.43 s


### Threads

A thread is an execution context, which is all the information a CPU needs to execute a stream of instructions.

Switching context every sys.getswitchinterval() (5ms default)

In [42]:
import sys


print(sys.getswitchinterval())

0.005


In [43]:
%%time

from functools import partial
from threading import Thread


thread_1 = Thread(target=partial(job, n=1))
thread_2 = Thread(target=partial(job, n=2))

thread_1.start()
thread_2.start()

thread_1.join()
thread_2.join()

--- request 1 sent
--- request 2 sent
--- response 1 received
--- response 2 received
CPU times: user 37.3 ms, sys: 14.2 ms, total: 51.5 ms
Wall time: 1.09 s


### Async

In [44]:
async def ajob(client, n):
    print(f"--- request {n} sent")
    await client.get(f"https://example.com/{n}")
    print(f"--- response {n} received")

In [47]:
import asyncio
import time


async def amain():
    async with httpx.AsyncClient() as client:
        await asyncio.gather(
            ajob(client=client, n=1),
            ajob(client=client, n=2)
        )


start = time.time()
await amain()
print(time.time() - start)

--- request 1 sent
--- request 2 sent
--- response 1 received
--- response 2 received
0.8680598735809326


## Why do we need async?

Cons of threads:
- threads are heavier
- switching is not under our control (*)

Cons of async:
- special syntax (steeper learning curve)
- need ioloop to execute
- no blocking code allowed (*)

## Timeline



- 1991 - The first python release
- 2001 - Simple generators ([PEP-255](https://peps.python.org/pep-0255/))
- 2002 - Twisted - event driven networking engine
- 2005 - Generator based coroutines
- 2008 - Python 3.0
- 2009 - Tornado opensourced by Facebook (developed by FriendFeed)
- 2012 - Python 3.3, proposed to make Tulip/asyncio a part of stdlib (`*`)
- 2014 - Python 3.4, asyncio is a part of stdlib
- 2015 - async/await syntax
- 2016 - Python 3.6
- 2018 - Python 3.7, support for [generator-based coroutines is deprecated](https://docs.python.org/3.7/library/asyncio-task.html#generator-based-coroutines) and is scheduled for removal in Python 3.10, FastAPI first release, tornado integration with asyncio by default
- 2021 - Python 3.10

`*` The Tulip project is the asyncio module for Python 3.3.

### asyncio success

- A part of stdlib
- Unified ioloop interface
- Native coroutines and async/await syntax
- Wide support

## From callbacks to async

### Synchronous

In [ ]:
from tornado.httpclient import HTTPClient


def synchronous_fetch(url):
    http_client = HTTPClient()
    response = http_client.fetch(url)
    return response.body

### With callbacks

In [ ]:
from tornado.httpclient import AsyncHTTPClient


def asynchronous_fetch_callbacks(url, callback):
    http_client = AsyncHTTPClient()

    def handle_response(response):
        callback(response.body)

    http_client.fetch(url, callback=handle_response)

### With `Future`

In [ ]:
from tornado.concurrent import Future
from tornado.httpclient import AsyncHTTPClient


def asynchronous_fetch_future(url):
    http_client = AsyncHTTPClient()
    my_future = Future()
    fetch_future = http_client.fetch(url)

    def on_fetch(f):
        my_future.set_result(f.result().body)

    fetch_future.add_done_callback(on_fetch)
    return my_future

### With Tornado `gen` (generator based coroutine)

In [ ]:
from tornado import gen
from tornado.httpclient import AsyncHTTPClient


@gen.coroutine
def asynchronous_fetch_gen(url):
    http_client = AsyncHTTPClient()
    response = yield http_client.fetch(url)
    raise gen.Return(response.body)

![coroutine yield](cotoutine_yield.png)

### `yield from` and `return` for generator bases coroutines

In [ ]:
@asyncio.coroutine
def asynchronous_fetch_gen(url):
    http_client = AsyncHTTPClient()
    response = yield http_client.fetch(url)
    return response.body


@asyncio.coroutine
def amain():
    result = yield from asynchronous_fetch_gen('https://example.com')
    return result

### With `async`/`await` (native coroutine)

In [ ]:
from tornado.httpclient import AsyncHTTPClient


async def asynchronous_fetch(url):
    http_client = AsyncHTTPClient()
    response = await http_client.fetch(url)
    return response.body

## Generators and coroutines



## New concepts

- async def (native coroutine)
- async for (async iterator, `__next__` -> `__anext__`)
- async with (async context manager, `__extent__` and `__exit__` -> `__aextent__` and `__aexit__`)
- await and awaitables

Awaitable:
- Coroutine
- Future
- Task (a subclass of Future)

await can be used only inside a coroutine.

In [ ]:
import asyncio

async def coro():
    await asyncio.sleep(0.5)

asyncio.run(coro())

TODO:
- generators and coroutines detour
- 